# microsoftmt
> Library to use Microsoft's machine translation services

In [ ]:
#| default_exp microsoftmt

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    homedir = os.getenv('HOME')
    GITHUB_TEST_FOLDER = homedir+"/github/polyglottech/mteval"
    %cd {GITHUB_TEST_FOLDER}
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests, uuid
import os
from dotenv import load_dotenv

class microsofttranslate:
    """
    Class to get translations from the Microsoft Translator API
    """
    def __init__(self):
        """
        Constructor of microsofttranslate class

        This will load the credentials for the Microsoft Translator API from the environment
        """
        running_in_colab = 'google.colab' in str(get_ipython())
        homedir = ""
        if running_in_colab:
            from google.colab import drive
            drive.mount('/content/drive')
            homedir = "/content/drive/MyDrive"
            # Colab doesn't have a mechanism to set environment variables other than python-dotenv
            ENV_FILE = homedir+'/secrets/.env'
            load_dotenv(ENV_FILE)
        self._subscription_key = os.getenv('MS_SUBSCRIPTION_KEY')
        self._region = os.getenv('MS_REGION')

    def translate_text(self,source, target, text):
        """
        Function to translate text into the target language

        Parameters:
            source (string): source language identifier as an ISO 639-1 language code. See https://g.co/cloud/translate/v2/translate-reference#supported_languages
            target (string): target language identifier as an ISO 639-1 language code. See https://g.co/cloud/translate/v2/translate-reference#supported_languages
            text (string): Source text to be translated

        Returns:
            hypothesis: Hypothesis translation from Google Translate
        
        """
        # Add your subscription key and endpoint
        subscription_key = self._subscription_key
        endpoint = "https://api.cognitive.microsofttranslator.com"

        # Add your location, also known as region. The default is global.
        # This is required if using a Cognitive Services resource.
        location = self._region

        path = '/translate'
        constructed_url = endpoint + path

        params = {
            'api-version': '3.0',
            'from': source,
            'to': [target]
        }
        constructed_url = endpoint + path

        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Ocp-Apim-Subscription-Region': location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }

        # You can pass more than one object in body.
        body = [{
            'text': text
        }]

        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()

        return response[0]["translations"][0]["text"]

In [ ]:
#| hide

msmt = microsofttranslate()
translation = msmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una macchina fantastica!"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()